# Python for (open) Neuroscience

_Lecture 1.1_ - More on `numpy`

Luigi Petrucco

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec-2025/blob/main/lectures/Lecture1.1_Numpy.ipynb)

## Lecture outline

- Transforming and combining arrays (and practicals)
- Operations with arrays (and practicals)
- Searching and sorting arrays

## Transforming and combining arrays

### `.T`

We can get a **transposed view** (NOT A COPY) of a matrix with the `.T` attribute:

In [9]:
import numpy as np

m = np.random.randint(0, 10, (3,2))
print(m)

[[2 9]
 [1 0]
 [6 1]]


In [10]:
m_t = m.T.copy()
m_t[0, 0] = 100
print(m)

[[2 9]
 [1 0]
 [6 1]]


In [ ]:
m_t[0, 0] = 2  # this is a view: if we change values in the transposed array, we change the original as well:
m

### `.flatten()`

We can flatten all values of an N-dimensional array into a 1D array with the `.flatten()` syntax. This will make **a copy of the array**!

In [20]:
m = np.random.randint(0, 10, (3,2, 4, 5))

### `np.concatenate()`

We can concatenate arrays along any dimension by putting them in a list and pass the list to the `np.concatenate()` function:

In [21]:
arr_list = [np.zeros(3), np.ones(3)]
print(arr_list)
np.concatenate(arr_list) 

[array([0., 0., 0.]), array([1., 1., 1.])]


array([0., 0., 0., 1., 1., 1.])

By default, we concatenate over the first dimension:

In [34]:
arr_list = [np.zeros((2,4, 5)), np.ones((2,3, 5))]
arr_list

[array([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]),
 array([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],
 
        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])]

In [35]:
np.concatenate(arr_list, axis=1)  # if ndims > 1 by default we concatenate over the first dimension

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]])

but we can pass an `axis` argument to change the default behavior:

In [ ]:
arr_list = [np.zeros((3,2)), np.ones((3,2))]
arr_list
np.concatenate(arr_list, axis=1) 

### `np.stack()`

We can pile up arrays over a new dimension with  `np.stack()`:

In [40]:
arr_list = [np.zeros((3,5)), np.ones((3,5))]
#arr_list
stacked = np.stack(arr_list)
print(stacked.shape)
print(stacked)

(2, 3, 5)
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]]


## Array operations

We obviously want to do some math with those arrays!

### Operations with numbers

Operations with arrays are **by default element-wise**! (performed separately on every number of the array)

Sum / subtraction / multiplication / division apply to individual entries of the array:

In [42]:
np.ones(3) + 1

array([2., 2., 2.])

In [44]:
my_arr = np.ones((4,3))
my_arr[0, :] = my_arr[0, :]*100 
my_arr

array([[100., 100., 100.],
       [  1.,   1.,   1.],
       [  1.,   1.,   1.],
       [  1.,   1.,   1.]])

Exponentiation also works element-wise:

In [45]:
np.array((1,2,3))**3

array([ 1,  8, 27])

### Operations between arrays

`numpy` works element-wise also when operating between arrays:

In [48]:
a1 = np.array([2, 3, 4])
a2 = np.array([1, 1, 1])
print(a1, a2)

a1 - a2

[2 3 4] [1 1 1]


array([1, 2, 3])

In [49]:
arr_1 = np.array([[1,2],
                  [3,4]])
arr_2 = np.array([[0,0],
                  [0,2]])

arr_1 * arr_2

array([[0, 0],
       [0, 8]])

In [50]:
arr_1 ** arr_2

array([[ 1,  1],
       [ 1, 16]])

Therefore, we normally expect arrays of matching shapes, or we get a `ValueError`!

In [51]:
np.ones((2, 3)) * np.ones((4, 5))

ValueError: operands could not be broadcast together with shapes (2,3) (4,5) 

Practicals 1.1.0

### Broadcasting

`numpy` has a smart way of dealing with some scenarios of non-matching dimensions, and we should use it!

Can be a bit tricky at the beginning, but it is very important: we can write very efficient and readable code with it!

In [60]:
# Assume we have a matrix of data:
data = np.array([[ 0.0,  0.0,  0.0],
                 [10.0, 10.0, 10.0],
                 [20.0, 20.0, 20.0],
                 [30.0, 30.0, 30.0]])

data.shape

(4, 3)

In [64]:
offsets = np.array([100, 200, 300])  # we want to add an offset to each data column:
print(offsets.shape)
data - offsets

(3,)


array([[-100., -200., -300.],
       [ -90., -190., -290.],
       [ -80., -180., -280.],
       [ -70., -170., -270.]])

### What is happening?

Numpy automatically infer missing values to create arrays of matching shape, where it can the operate element-wise!

![Alt Text](https://numpy.org/doc/stable/_images/broadcasting_2.png)

## How does broadcasting work

When operating on two arrays, NumPy compares their shapes. It starts **with the trailing** dimension - i.e., the rightmost dimension in the `shape` tuple - and works its way left. In a `(2,3)` matrix, broadcasting will start from columns (the `3` in `(2,3)`) and move on to rows (the `2` in `(2,3)`).

Two dimensions are compatible when:

 - they are equal, or
 - one of them is 1.

![Alt Text](https://i0.wp.com/andrewm4894.com/wp-content/uploads/2020/10/Annotation-2020-10-15-133235.jpg?w=486&ssl=1)

In our case:

In [65]:
print(f"shape a: {data.shape}")
print(f"shape b: {offsets.shape}")

shape a: (4, 3)
shape b: (3,)


Shape b matches shape a over the last dimension, and is propagated over the rest of the dimensions

For example, this operation will not work!

In [66]:
a = np.concatenate([np.full((1,4), i) for i in range(5)], axis=0)
b = np.ones(5)
print(a, b)

[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]] [1. 1. 1. 1. 1.]


In [67]:
print("shape a: ", a.shape)
print("shape b: ", b.shape)

# this will not work as the rightmost dimensions are 5 and 4:
a + b 

shape a:  (5, 4)
shape b:  (5,)


ValueError: operands could not be broadcast together with shapes (5,4) (5,) 

To make it work, we can use a trick: add a new "dummy" singleton dimension to `b` that will be broadcasted with the syntax `[:, np.newaxis]`

In [72]:
b[:, np.newaxis]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [69]:
b_twodim = b[:, np.newaxis]  # This does the trick by adding a dummy singleton dimension
print("shape a: ", a.shape)
print("shape b_twodim: ", b_twodim.shape)
a + b_twodim  # now the last dimension is compatible between the two arrays:

shape a:  (5, 4)
shape b_twodim:  (5, 1)


array([[1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.],
       [4., 4., 4., 4.],
       [5., 5., 5., 5.]])

Practicals 1.1.1

## More on `numpy` functions

### `np.mean()` / `np.nanmean()`

Calculate the average of an array, either global or along some axis:

In [74]:
import numpy as np
arr = np.random.randint(0, 255, (5, 6))
mean = np.mean(arr)  # compute the mean of the whole array

print(f"{arr};\nmean: {mean}")

[[ 70 237 122  55 129   0]
 [226 172 166 152  58 206]
 [ 24  28 111 228 114 121]
 [144  14  74  36 251 113]
 [ 89 130  83 233 215  86]];
mean: 122.9


If we want to take the average along a specific dimension, we can pass the axis as a parameter:

In [79]:
import numpy as np
arr = np.random.randint(0, 255, (3, 2, 2))
arr_mean = np.mean(arr, axis=(1, 2))  # we specify one axis along which to mean

print(f"{arr};\nmean: {arr_mean}")

[[[161 181]
  [133 210]]

 [[209 177]
  [186 220]]

 [[120   1]
  [  7 226]]];
mean: [171.25 198.    88.5 ]


If there are nan values around, we have to use `np.nanmean()`:

In [80]:
import numpy as np

# we need a float dtype to use nan values:
arr = np.random.randint(0, 255, (5, 6)).astype(float) 

arr[0, 0] = np.nan
arr_mean = np.mean(arr)  # regular mean
arr_nan_mean = np.nanmean(arr)  # use nanmean

print(f"{arr};\nregular mean: {arr_mean}\nnanmean: {arr_nan_mean}")

[[ nan 198.  12. 138.  84.   7.]
 [ 28.  60. 203. 140. 146. 205.]
 [230.  84.  70. 179. 128. 222.]
 [204. 166.  82.  26. 213. 123.]
 [165. 174. 184. 182. 248. 160.]];
regular mean: nan
nanmean: 140.0344827586207


Many of the functions we're about to see behave in this way - assume they have a nan-dealing equivalent!

 - `np.std()` / `np.nanstd()`
 - `np.percentile()` / `np.nanpercentile()`
 - `np.max()` / `np.nanmax()`
 - ...

### `np.std()` / `np.nanstd()`

Calculate the standard deviation of an array, either global or along some axis:

In [83]:
arr = np.random.normal(0, 3, 1000)
np.std(arr)

np.float64(3.054255688975187)

### `np.median()` / `np.nanmedian()`

Calculate the median of an array, either global or along some axis:

In [ ]:
arr = np.random.randint(0, 100, (1000, 10))
np.median(arr, axis=0)

### `np.max()` / `np.min()`

Calculate the minimum or maximum of an array, either global or along some axis:

In [ ]:
arr = np.random.randint(0, 100, 1000)

np.min(arr), np.max(arr)  # print min and max together

### `np.percentile()`

Calculate a given percentile of an array, either global or along some axis:

In [84]:
arr = np.random.randint(0, 1000, 10000)

np.percentile(arr, 75)  # print min and max together

np.float64(742.0)

### `np.unique()`

Return unique values of an array, and if asked their counts

In [90]:
np.unique(np.array([1, 2, 2, 2, 3, 3, 3, 3]), return_counts=True)

(array([1, 2, 3]), array([1, 3, 4]))

In [ ]:
# If we ask we can get counts as well
unique_values, counts = np.unique([1,2,2,3], return_counts=True) 

print("unique:", unique_values)§

print("counts:", counts)

In [ ]:
arr = np.random.normal(0, 1, 1000000)

### `np.diff()` / `np.cumsum()`  

We can compute cumulative sums (integrals) of an array with `np.cumsum()`:

In [93]:
my_arr = np.array([[1,2,3,4],
                   [1,2,4,5]])
np.cumsum(my_arr, axis=1)

array([[ 1,  3,  6, 10],
       [ 1,  3,  7, 12]])

We can compute differences between consecutive elements of an array using `np.diff()`:

In [94]:
my_arr = np.array([1,2,3,5])
np.diff(my_arr)

array([1, 1, 2])

(Practicals 1.1.2)

### Write code the `numpy` way

When operating with matrices, you should always aim at writing <span style="color:indianred">vectorized</span> code

Vectorized code: code where for loops are replaced by operations over matrix dimensions

An very simple example: vectors multiplication

In [96]:
vector_1 = np.random.normal(0, 1, (10000000,))
vector_2 = np.random.normal(0, 1, (10000000,))

In [97]:
%%timeit
product = np.zeros(vector_1.shape)  # initialize empty result vector

# Compute the multiplication in a loop:
for i in range(vector_1.shape[0]):
    product[i] = vector_1[i] * vector_2[i]

1.36 s ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
%%timeit
product = vector_1 * vector_2

8.1 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Another example: Z-score the rows of a matrix:

In [99]:
data_matrix = np.random.randint(0, 255, (100000, 100))

In [100]:
%%timeit
normalized_matrix = np.zeros(data_matrix.shape)  # start an empty matrix of matching shape 

# Loop over rows (first dimension), take mean and std, subtract and divide:
for i in range(data_matrix.shape[0]):
    row_mean = np.mean(data_matrix[i, :])
    row_std = np.std(data_matrix[i, :])
    
    normalized_matrix[i, :] = (data_matrix[i, :] - row_mean) / row_std

1.1 s ± 7.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
%%timeit
rows_mean = np.mean(data_matrix, axis=1)  # vectorized mean
rows_std = np.std(data_matrix, axis=1)  # vectorized std

# Now we write the normalization as a vector operation.
# Note how we use broadcasting to make sure the right dimensions are propagated!
normalized_matrix = data_matrix - rows_mean[:, np.newaxis] / rows_std[:, np.newaxis]

42.1 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
